In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from time import sleep

In [11]:
TIME_RESOLUTION = 5 # seconds

In [12]:
data = pd.read_csv("k3s_resource_usage.csv").drop(columns=["node"])

In [13]:
def write_image(fig, fname):
    fig.show()

    # create random figure to load math js
    fig2 = px.scatter(x=[0, 1], y=[0, 1])
    fig2.write_image(fname)
    sleep(1)

    fig.write_image(fname)

In [14]:
# change units and convert to int/float
data["cpu"] = data["cpu"].str.replace("m", "").astype(int)
data["memory"] = data["memory"].str.replace("Mi", "").astype(int)
data["cpu_per"] = data["cpu_per"].str.replace("%", "").astype(float) / 100
data["memory_per"] = data["memory_per"].str.replace("%", "").astype(float) / 100

data.head()

,timestamp,cpu,cpu_per,memory,memory_per
0,2025-04-23T12:09:09.364298,572,0.07,5122,0.42
1,2025-04-23T12:09:09.364298,407,0.05,3980,0.50
2,2025-04-23T12:09:09.364298,42,0.00,1696,0.21
3,2025-04-23T12:09:09.364298,33,0.00,1701,0.21
4,2025-04-23T12:09:09.364298,29,0.00,1788,0.22


In [15]:
data = data.groupby("timestamp").agg(
    {
        "cpu": "sum",
        "cpu_per": "mean",
        "memory": "sum",
        "memory_per": "mean"
    }
).reset_index()

data = data.iloc[2:].reset_index(drop=True)
data["time"] = data.index * TIME_RESOLUTION

data.head()

,timestamp,cpu,cpu_per,memory,memory_per,time
0,2025-04-23T12:09:20.756687,1157,0.024,14247,0.31,0
1,2025-04-23T12:09:26.548629,1127,0.024,14234,0.31,5
2,2025-04-23T12:09:32.310791,1127,0.024,14234,0.31,10
3,2025-04-23T12:09:38.011474,1127,0.024,14234,0.31,15
4,2025-04-23T12:09:43.719487,1242,0.026,14222,0.31,20


In [16]:
# horizontal lines
line_x = {"MinIO / TimescaleDB": 12, "Data Collector": 48, "Inference Service": 56, "Stress Test": 64}

for k, v in line_x.items():
    line_x[k] = v - 2

In [17]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data["time"],
        y=data["cpu"],
        mode="lines",
        name="CPU Usage",
        line=dict(color="blue"),
    )
)

rectangular_x_start = [0] + list(line_x.values()) + [data["time"].max() / TIME_RESOLUTION]
colours = ["blue", "green", "orange", "purple", "red"]

for i in range(len(rectangular_x_start) - 1):
    fig.add_shape(
        type="rect",
        x0=rectangular_x_start[i] * TIME_RESOLUTION,
        x1=rectangular_x_start[i + 1] * TIME_RESOLUTION,
        y0=data["cpu"].min(),
        y1=data["cpu"].max() + 1e3,
        fillcolor=colours[i % len(colours)],
        opacity=0.08,
        layer="below",
        line_width=0,
    )

# add horizontal lines
for x_name, x in line_x.items():
    fig.add_trace(
        go.Scatter(
            x=[x*TIME_RESOLUTION, x*TIME_RESOLUTION],
            y=[data["cpu"].min(), data["cpu"].max()-1e3],
            mode="lines",
            line=dict(color="red", dash="dash"),
        )
    )

    fig.add_annotation(
        x=x*TIME_RESOLUTION,
        y=data["cpu"].max()-1e3,
        text=x_name,
        showarrow=False,
        yshift=5,
        font=dict(color="red")
    )

fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="CPU Usage (m)",
    height=400,
    width=1200,
    margin=dict(l=20, r=20, t=20, b=20),
    showlegend=False,
    yaxis=dict(
        range=[data["cpu"].min()-50, data["cpu"].max()+250],
    ),
)

write_image(fig, "deploy_cpu_usage.pdf")
# write_image(fig, "deploy_cpu_usage.png")

In [18]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data["time"],
        y=data["memory"],
        mode="lines",
        line=dict(color="blue"),
    )
)

rectangular_x_start = [0] + list(line_x.values()) + [data["time"].max() / TIME_RESOLUTION]
colours = ["blue", "green", "orange", "purple", "red"]

for i in range(len(rectangular_x_start) - 1):
    fig.add_shape(
        type="rect",
        x0=rectangular_x_start[i] * TIME_RESOLUTION,
        x1=rectangular_x_start[i + 1] * TIME_RESOLUTION,
        y0=data["cpu"].min(),
        y1=data["cpu"].max() + 1e4,
        fillcolor=colours[i % len(colours)],
        opacity=0.08,
        layer="below",
        line_width=0,
    )

# add horizontal lines
for x_name, x in line_x.items():
    fig.add_trace(
        go.Scatter(
            x=[x*TIME_RESOLUTION, x*TIME_RESOLUTION],
            y=[data["memory"].min(), data["memory"].max() - 3e2],
            mode="lines",
            line=dict(color="red", dash="dash"),
        )
    )

    fig.add_annotation(
        x=x*TIME_RESOLUTION,
        y=data["memory"].max() - 3e2,
        text=x_name,
        showarrow=False,
        yshift=5,
        font=dict(color="red")
    )

fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="Memory usage (MiB)",
    height=400,
    width=1200,
    margin=dict(l=20, r=20, t=20, b=20),
    showlegend=False,
    yaxis=dict(
        range=[data["memory"].min()-10, data["memory"].max()+70],
    ),
)
write_image(fig, "deploy_ram_usage.pdf")
# write_image(fig, "deploy_ram_usage.png")